In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, f1_score

In [2]:
data = pd.read_csv('titanic.csv')

In [3]:
# select relevant features (columns) for training
features = ["pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]
target = "survived"

# handle missing values
data = data[features + [target]].dropna()

X = data[features]
y = data[target]

## Preprocessing

In [4]:
# transform binary categorical variables to 1/0
X["sex"] = (X["sex"] == "female").astype(int)

# transform categorical variables to matrix
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['embarked'])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

/var/folders/9r/8xlgm3yj4gj9dndpdbqg4xd00000gn/T/ipykernel_3901/1837508742.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["sex"] = (X["sex"] == "female").astype(int)


In [5]:
# split data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [6]:
# normalize features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

In [7]:
ann = tf.keras.models.Sequential()

# add the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

# add hidden layers
for _ in range(3):
    ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# add the output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Training the ANN

In [8]:
# compile the ANN
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# train the ANN on the training set
ann.fit(X_train, y_train, batch_size=2, epochs=100)

Epoch 1/100


2024-05-12 21:30:48.135491: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


417/417 [==============================] - 0s 387us/step - loss: 0.6693 - accuracy: 0.6199
Epoch 2/100
417/417 [==============================] - 0s 356us/step - loss: 0.6059 - accuracy: 0.7074
Epoch 3/100
417/417 [==============================] - 0s 329us/step - loss: 0.5258 - accuracy: 0.7614
Epoch 4/100
417/417 [==============================] - 0s 329us/step - loss: 0.4817 - accuracy: 0.7698
Epoch 5/100
417/417 [==============================] - 0s 327us/step - loss: 0.4629 - accuracy: 0.7806
Epoch 6/100
417/417 [==============================] - 0s 328us/step - loss: 0.4551 - accuracy: 0.7854
Epoch 7/100
417/417 [==============================] - 0s 328us/step - loss: 0.4508 - accuracy: 0.7950
Epoch 8/100
417/417 [==============================] - 0s 330us/step - loss: 0.4429 - accuracy: 0.7998
Epoch 9/100
417/417 [==============================] - 0s 330us/step - loss: 0.4425 - accuracy: 0.8034
Epoch 10/100
417/417 [==============================] - 0s 333us/step - loss: 0.4412 

Epoch 80/100
417/417 [==============================] - 0s 384us/step - loss: 0.4050 - accuracy: 0.8249
Epoch 81/100
417/417 [==============================] - 0s 373us/step - loss: 0.4037 - accuracy: 0.8309
Epoch 82/100
417/417 [==============================] - 0s 382us/step - loss: 0.4063 - accuracy: 0.8321
Epoch 83/100
417/417 [==============================] - 0s 360us/step - loss: 0.4037 - accuracy: 0.8261
Epoch 84/100
417/417 [==============================] - 0s 359us/step - loss: 0.4027 - accuracy: 0.8309
Epoch 85/100
417/417 [==============================] - 0s 364us/step - loss: 0.4030 - accuracy: 0.8249
Epoch 86/100
417/417 [==============================] - 0s 362us/step - loss: 0.4059 - accuracy: 0.8225
Epoch 87/100
417/417 [==============================] - 0s 365us/step - loss: 0.4066 - accuracy: 0.8249
Epoch 88/100
417/417 [==============================] - 0s 340us/step - loss: 0.4000 - accuracy: 0.8285
Epoch 89/100
417/417 [==============================] - 0s 327us

In [10]:
# evaluate the model on test data
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5)

# accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# f1 score
f1_score = f1_score(y_test, y_pred)
print(f'F1 Score: {f1_score}')

7/7 [==============================] - 0s 429us/step
Accuracy: 0.8181818181818182
Precision: 0.8
F1 Score: 0.759493670886076


In [11]:
# convert ANN model to TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(ann)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/9r/8xlgm3yj4gj9dndpdbqg4xd00000gn/T/tmpa94lww1h/assets


2024-05-12 21:31:03.396932: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-05-12 21:31:03.396945: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-05-12 21:31:03.397270: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/9r/8xlgm3yj4gj9dndpdbqg4xd00000gn/T/tmpa94lww1h
2024-05-12 21:31:03.397960: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-05-12 21:31:03.397964: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/9r/8xlgm3yj4gj9dndpdbqg4xd00000gn/T/tmpa94lww1h
2024-05-12 21:31:03.400237: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2024-05-12 21:31:03.400875: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-05-12 21:31:03.429908: I tensorflow/cc/saved_model/loader.

In [12]:
with open('ann_model.tflite', 'wb') as f:
    f.write(tflite_model)